### Mapping table associating awards to works, based on backfill table `openalex.mid.work_funder`

In [0]:
%sql
CREATE OR REPLACE TABLE openalex.awards.work_awards
USING delta
AS
WITH work_funders as (
  SELECT
    paper_id,
    funder_id,
    funder_award_id
  FROM openalex.mid.work_funder
  LATERAL VIEW EXPLODE(award_ids) AS funder_award_id
),
awards as (
  SELECT
    id,
    display_name,
    funder_id,
    funder_award_id,
    funder
  FROM openalex.awards.openalex_awards
),
back_fill_awards as (
  SELECT
    wf.paper_id as work_id,
    struct(
      concat("https://openalex.org/G", a.id) as id,
      a.display_name,
      a.funder_award_id,
      a.funder.id as funder_id,
      a.funder.display_name as funder_display_name
    ) as award,
    1 as priority -- Prefer existing data
  FROM
    work_funders wf
    JOIN
      awards a
    ON
      wf.funder_id = a.funder_id AND
      wf.funder_award_id = a.funder_award_id
),
gtr_awards as (
  SELECT
    work_id,
    struct(
      concat("https://openalex.org/G", abs(xxhash64(id)) % 9000000000) as id,
      NULL as display_name,
      funder_award_id,
      funder.id as funder_id, 
      funder.display_name as funder_display_name
    ) as award,
    2 as priority
  FROM openalex.awards.gtr_awards
  WHERE work_id IS NOT NULL
),
combined_awards AS (
  SELECT * FROM back_fill_awards
  UNION ALL
  SELECT * FROM gtr_awards
)

SELECT
  work_id,
  award
FROM combined_awards
QUALIFY ROW_NUMBER() OVER (
    PARTITION BY work_id, award.id 
    ORDER BY priority ASC
) = 1